# Prepare excel read with pandas

In [136]:
import pandas as pd
# data = pd.read_excel(r'monthly_market_data - Copy.xlsx')
data = pd.read_excel(r'monthly market data.xlsx')

# print(data)


# Read Excel to memnory and create classes

In [137]:
import operator

class CompanyInfo:
    def __init__(self, tickerKey, name, month,numberOfOutstandingShares, marketcap,b2m, adjClose,inv,op,sourceSubsectorCode
):
        self.tickerKey = tickerKey
        self.name = name
        self.month = month
        self.rollingAverage = None
        self.numberOfOutstandingShares = numberOfOutstandingShares if not pd.isnull(numberOfOutstandingShares) else None
        self.marketcap = marketcap if not pd.isnull(marketcap) else None
        self.b2m = b2m if not pd.isnull(b2m) else None
        self.closeAdjust = adjClose
        self.momentum = None
        self.sizeMOM = None
        self.sizeHML = None
        self.sizeRMW = None
        self.sizeCMA = None
        self.CMASize = None
        self.RMWSize = None
        self.BperMSize = None
        self.currentYield = None
        self.inv = inv if not pd.isnull(inv) else None
        self.op = op if not pd.isnull(op) else None
        self.sourceSubsectorCode = sourceSubsectorCode


    def __repr__(self):
         return self.__str__()

    def __str__(self):
        return "tickerKey:"+str(self.tickerKey) + \
            "\t month:" + str(self.month) + \
            "\t closeAdjust:" + str(self.closeAdjust) + \
            "\t currentYield:" + str(self.currentYield) + \
            "\t numberOfOutstandingShares:" + str(self.numberOfOutstandingShares) + \
            "\t marketcap:" + str(self.marketcap) + \
            "\t b2m:" + str(self.b2m) + \
            "\t rollingAverage:" + str(self.rollingAverage) + \
            "\t momentum:" + str(self.momentum) + \
            "\t sizeMOM:" + str(self.sizeMOM) + \
            "\t sizeHML:" + str(self.sizeHML) + \
            "\t sizeRMW:" + str(self.sizeRMW) + \
            "\t sizeCMA:" + str(self.sizeCMA) +"\n"

    def convertToList(self):
      return [getattr(self,atr) for atr in self.__dict__]


companyInfoDict  = {}
def addCompanyInfoToList(companyInfo:CompanyInfo):
    if companyInfo.tickerKey not in companyInfoDict:
      companyInfoDict[companyInfo.tickerKey]={}
    if companyInfo.month not in companyInfoDict[companyInfo.tickerKey]:
      companyInfoDict[companyInfo.tickerKey][companyInfo.month]=companyInfo
    else:
      raise Exception("Company info exists." + str(companyInfo))

def findCompanyInfo(tickerKey, month):
    if tickerKey in companyInfoDict and month in companyInfoDict[tickerKey]:
      return companyInfoDict[tickerKey][month]
    return None

def displayCompanyInfoHeader():
  for tickerKey in companyInfoDict.keys():
      for month in companyInfoDict[tickerKey].keys():
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        attrs = []
        for attribute in vars(company):
          attrs.append(attribute)
        print(attrs)
        return

def displayCompanyInfo(tickerKeys:list,months:list):
    displayCompanyInfoHeader()
    for tickerKey in tickerKeys:
      sortedMonths = sorted(months)
      for month in sortedMonths:
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        print(company.convertToList())

allMonths = []


for i, companyInfoPandas in data.iterrows():
    ci = CompanyInfo(tickerKey=companyInfoPandas["TickerKey"],
                                       name=companyInfoPandas["TickerNamePooyaFA"],
                                       month=companyInfoPandas["DayKeyFA"]// 100,
                                       numberOfOutstandingShares=companyInfoPandas["NumberOfOutstandingShares"],
                                       marketcap=companyInfoPandas["marketcap"],
                                       b2m=companyInfoPandas["B2M"],
                                       adjClose=companyInfoPandas["AdjClose"],
                                       inv=companyInfoPandas["INV"],
                                       op=companyInfoPandas["OP"],
                                       sourceSubsectorCode=companyInfoPandas["SourceSubsectorCode"],
                                       )
    addCompanyInfoToList(ci)
    allMonths.append(ci.month)

allMonths = list(set(allMonths))
allMonths.sort()


In [138]:
# companies to excel
def exportCompanyListToExcel(excelFileName):
  rows = []
  for company in companyInfoDict.keys():
    for month in sorted(companyInfoDict[company].keys()):
      rows.append(companyInfoDict[company][month].convertToList())

  company_ = None
  for tickerKey in companyInfoDict.keys():
      for month in sorted(companyInfoDict[company].keys()):
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        company_ = company
        break
      if company_ is not None:
        break
  dfOut = pd.DataFrame(rows,columns=company_.__dict__)
  dfOut.to_excel(excelFileName+'.xlsx', sheet_name='TMPExport')

# Dict access sample and test

In [139]:
print(list(companyInfoDict.keys())[:10])
print(list(companyInfoDict[1].keys())[:10])
print(companyInfoDict[1][139712])


[1, 9, 10, 219, 223, 231, 234, 239, 243, 247]
[139712, 139801, 139802, 139803, 139804, 139805, 139806, 139807, 139808, 139809]
tickerKey:1	 month:139712	 closeAdjust:802.5232992867204	 currentYield:None	 numberOfOutstandingShares:3000000000.0	 marketcap:4959000000000.0	 b2m:1.2906721802	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None



# Extract company names

In [140]:
companyTickerSet = list(companyInfoDict.keys())
print(companyTickerSet)


[1, 9, 10, 219, 223, 231, 234, 239, 243, 247, 251, 255, 259, 263, 267, 271, 275, 279, 283, 287, 291, 295, 301, 303, 307, 311, 315, 320, 323, 327, 336, 340, 344, 347, 350, 355, 359, 363, 370, 375, 378, 383, 389, 395, 398, 403, 406, 411, 414, 422, 423, 430, 432, 434, 438, 446, 447, 450, 454, 458, 463, 466, 474, 475, 478, 483, 487, 490, 494, 499, 502, 510, 511, 515, 517, 526, 527, 534, 538, 547, 550, 558, 559, 563, 566, 574, 575, 578, 583, 587, 591, 599, 603, 607, 610, 615, 622, 627, 631, 635, 640, 646, 647, 654, 655, 659, 662, 667, 670, 675, 680, 687, 689, 698, 708, 711, 718, 724, 725, 731, 733, 743, 744, 747, 751, 754, 759, 762, 767, 772, 774, 779, 786, 787, 791, 794, 798, 801, 807, 810, 815, 820, 822, 827, 839, 842, 847, 852, 855, 863, 866, 871, 875, 879, 883, 887, 890, 893, 899, 902, 908, 909, 915, 922, 931, 935, 939, 943, 947, 951, 955, 958, 965, 968, 969, 981, 985, 990, 993, 1003, 1010, 1013, 1020, 1028, 1032, 1033, 1041, 1044, 1048, 1053, 1059, 1064, 1068, 1074, 1077, 1084, 1088, 1

# Extract company months

In [141]:
print(allMonths)
print("Total:",len(allMonths))

[138001, 138002, 138003, 138004, 138005, 138006, 138007, 138008, 138009, 138010, 138011, 138012, 138101, 138102, 138103, 138104, 138105, 138106, 138107, 138108, 138109, 138110, 138111, 138112, 138201, 138202, 138203, 138204, 138205, 138206, 138207, 138208, 138209, 138210, 138211, 138212, 138301, 138302, 138303, 138304, 138305, 138306, 138307, 138308, 138309, 138310, 138311, 138312, 138401, 138402, 138403, 138404, 138405, 138406, 138407, 138408, 138409, 138410, 138411, 138412, 138501, 138502, 138503, 138504, 138505, 138506, 138507, 138508, 138509, 138510, 138511, 138512, 138601, 138602, 138603, 138604, 138605, 138606, 138607, 138608, 138609, 138610, 138611, 138612, 138701, 138702, 138703, 138704, 138705, 138706, 138707, 138708, 138709, 138710, 138711, 138712, 138801, 138802, 138803, 138804, 138805, 138806, 138807, 138808, 138809, 138810, 138811, 138812, 138901, 138902, 138903, 138904, 138905, 138906, 138907, 138908, 138909, 138910, 138911, 138912, 139001, 139002, 139003, 139004, 139005,

In [142]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'sizeMOM', 'sizeHML', 'sizeRMW', 'sizeCMA', 'CMASize', 'RMWSize', 'BperMSize', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000.0, 460800000000.0, 4.9385362299, 10.282307670219264, None, None, None, None, None, None, None, None, None, 0.8095322274960436, 0.29584800863123173, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000.0, 481500000000.0, 5.1603845371, 10.744208210092395, None, None, None, None, None, None, None, None, None, 0.9470740522344644, 0.12900261418635073, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000.0, 494910000000.0, 5.3041036578, 11.043439429401511, None, None, None, None, None, None, None, None, None, 0.3608801681067647, 0.8296664347356032, 4424]


# Fill missing close adjust

In [143]:
def calculateCloseAdjust(tickerKey, monthIndex):
  privuseMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex-1])
  if privuseMonthCompanyInfo is None:
    return None
  nextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+1])

  if nextMonthCompanyInfo is not None:
    return ((nextMonthCompanyInfo.closeAdjust / privuseMonthCompanyInfo.closeAdjust) ** (1/2)) * privuseMonthCompanyInfo.closeAdjust

  if monthIndex + 2 >= len(allMonths):
    return None
  secondNextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+2])
  if secondNextMonthCompanyInfo is None:
    return None
  return ((secondNextMonthCompanyInfo.closeAdjust / privuseMonthCompanyInfo.closeAdjust) ** (1/3)) * privuseMonthCompanyInfo.closeAdjust
  
def calculateMarketcapAdjust(tickerKey, monthIndex):
  privuseMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex-1])
  if privuseMonthCompanyInfo is None or privuseMonthCompanyInfo.marketcap is None:
    return None
  nextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+1])

  if nextMonthCompanyInfo is not None and nextMonthCompanyInfo.marketcap is not None:
    return ((nextMonthCompanyInfo.marketcap / privuseMonthCompanyInfo.marketcap) ** (1/2)) * privuseMonthCompanyInfo.marketcap

  if monthIndex + 2 >= len(allMonths):
    return None
  secondNextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+2])
  if secondNextMonthCompanyInfo is None or secondNextMonthCompanyInfo.marketcap is None:
    return None
  return ((secondNextMonthCompanyInfo.marketcap / privuseMonthCompanyInfo.marketcap) ** (1/3)) * privuseMonthCompanyInfo.marketcap
  
  # print(calculatedCloseAdjust)

_missingData = []
for tickerKey in companyTickerSet:
  # companyHistoricalData = [companyInfo for companyInfo in companyInfoList if companyInfo.tickerKey == tickerKey]
  startOfData = False
  for monthIndex in range(1,len(allMonths)-1):
    currentCompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if currentCompanyInfo is not None:
        startOfData = True
        continue
    if currentCompanyInfo is None:
      closeAdjust = calculateCloseAdjust(tickerKey,monthIndex)
      if closeAdjust is None: 
        if startOfData :
          _missingData.append(["closeAdjust for tick",tickerKey," Month",allMonths[monthIndex]," Is None"])
        continue
      privuseMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex-1])

      adjustedCompanyInfo = CompanyInfo(tickerKey,privuseMonthCompanyInfo.name,allMonths[monthIndex],None,None,None,None,None,None,privuseMonthCompanyInfo.sourceSubsectorCode)
      adjustedCompanyInfo.closeAdjust = closeAdjust
      # print(tickerKey,allMonths[month],privuseMonthCompanyInfo) # log before adjusted
      if privuseMonthCompanyInfo.numberOfOutstandingShares == None:
        print("None privuse marketcap" ,privuseMonthCompanyInfo)
        continue
      adjustedCompanyInfo.marketcap = calculateMarketcapAdjust(tickerKey,monthIndex)
      if adjustedCompanyInfo.marketcap is None:
        print("***\tNo market cap while having closed adjust for TickerKey:" +  str(adjustedCompanyInfo.tickerKey) + " Month:" + str(adjustedCompanyInfo.month))
        continue
      if privuseMonthCompanyInfo.numberOfOutstandingShares == None:
        print("None numberOfOutstandingShares" ,privuseMonthCompanyInfo)
        continue
      adjustedCompanyInfo.numberOfOutstandingShares = privuseMonthCompanyInfo.numberOfOutstandingShares
      addCompanyInfoToList(adjustedCompanyInfo)
      # print(tickerKey,allMonths[month]) # log adjusted


# print([companyInfo for companyInfo in _missingData if companyInfo[1] == 9])

None privuse marketcap tickerKey:2511	 month:138608	 closeAdjust:306.273415	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

None privuse marketcap tickerKey:2511	 month:139004	 closeAdjust:281.955124	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

None privuse marketcap tickerKey:2551	 month:138502	 closeAdjust:22.141126906876828	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

None privuse marketcap tickerKey:2551	 month:138708	 closeAdjust:83.16439779553711	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

*

In [144]:
print("{} of {}".format(len(_missingData),data.size))
print(_missingData[:10])

8336 of 1062292
[['closeAdjust for tick', 9, ' Month', 138403, ' Is None'], ['closeAdjust for tick', 9, ' Month', 138404, ' Is None'], ['closeAdjust for tick', 9, ' Month', 138405, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139307, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139308, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139309, ' Is None'], ['closeAdjust for tick', 231, ' Month', 138311, ' Is None'], ['closeAdjust for tick', 231, ' Month', 138312, ' Is None'], ['closeAdjust for tick', 231, ' Month', 138401, ' Is None'], ['closeAdjust for tick', 234, ' Month', 138907, ' Is None']]


# Test data

In [145]:

displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'sizeMOM', 'sizeHML', 'sizeRMW', 'sizeCMA', 'CMASize', 'RMWSize', 'BperMSize', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000.0, 460800000000.0, 4.9385362299, 10.282307670219264, None, None, None, None, None, None, None, None, None, 0.8095322274960436, 0.29584800863123173, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000.0, 481500000000.0, 5.1603845371, 10.744208210092395, None, None, None, None, None, None, None, None, None, 0.9470740522344644, 0.12900261418635073, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000.0, 494910000000.0, 5.3041036578, 11.043439429401511, None, None, None, None, None, None, None, None, None, 0.3608801681067647, 0.8296664347356032, 4424]


# Yield calculations

In [146]:
for tickerKey in companyTickerSet:
  for monthIndex in range(1,len(allMonths)):
    lastMonthData:CompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex-1])
    if lastMonthData is None:
      continue
    thisMonthData:CompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if thisMonthData is None:
      continue
    thisMonthData.currentYield = thisMonthData.closeAdjust / lastMonthData.closeAdjust - 1

In [147]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'sizeMOM', 'sizeHML', 'sizeRMW', 'sizeCMA', 'CMASize', 'RMWSize', 'BperMSize', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000.0, 460800000000.0, 4.9385362299, 10.282307670219264, None, None, None, None, None, None, None, None, None, 0.8095322274960436, 0.29584800863123173, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000.0, 481500000000.0, 5.1603845371, 10.744208210092395, None, None, None, None, None, None, None, None, 0.044921875, 0.9470740522344644, 0.12900261418635073, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000.0, 494910000000.0, 5.3041036578, 11.043439429401511, None, None, None, None, None, None, None, None, 0.027850467289719738, 0.3608801681067647, 0.8296664347356032, 4424]


# Calculate rolling average

In [148]:
def getRollingDataWindow(tickerKey,rollingWindowSize,endMonthIndex):
  rollingDataWindow = []
  for monthIndex in range(endMonthIndex+1 - rollingWindowSize,endMonthIndex+1):
    currentCompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if currentCompanyInfo is None:
        continue
    rollingDataWindow.append(currentCompanyInfo)
  return rollingDataWindow

In [149]:
def average(dataList):
    if len(dataList)==0:
        return None
    data = [info.currentYield for info in dataList]
    if None in data:
      return None
    # print(data)
    return sum(data) / len(dataList)

def valueWeightedAverage(dataList):
    if len(dataList)==0:
        return None
    lastMonth = allMonths[allMonths.index(dataList[0].month) - 1]
    data = [info.currentYield * findCompanyInfo(info.tickerKey,lastMonth).marketcap for info in dataList]
    marketcapSum = sum([findCompanyInfo(info.tickerKey,lastMonth).marketcap for info in dataList])
    if None in data:
      return None
    # print(data)
    return sum(data) / marketcapSum

# print(companyInfoList)
rollingAverageWindowSize = 6     #Can be change
for tickerKey in companyTickerSet:
    # print("tickerKey",tickerKey)
    for endMonthIndex in range(rollingAverageWindowSize-1,len(allMonths)):
      # print("endMonthIndex",endMonthIndex)
      rollingDataWindow = getRollingDataWindow(tickerKey, rollingAverageWindowSize,endMonthIndex)
      if len(rollingDataWindow) != rollingAverageWindowSize: continue
      currentCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey,allMonths[endMonthIndex])
      currentCompanyInfo.rollingAverage = average(rollingDataWindow)

In [150]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'sizeMOM', 'sizeHML', 'sizeRMW', 'sizeCMA', 'CMASize', 'RMWSize', 'BperMSize', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000.0, 460800000000.0, 4.9385362299, 10.282307670219264, None, None, None, None, None, None, None, None, None, 0.8095322274960436, 0.29584800863123173, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000.0, 481500000000.0, 5.1603845371, 10.744208210092395, None, None, None, None, None, None, None, None, 0.044921875, 0.9470740522344644, 0.12900261418635073, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000.0, 494910000000.0, 5.3041036578, 11.043439429401511, None, None, None, None, None, None, None, None, 0.027850467289719738, 0.3608801681067647, 0.8296664347356032, 4424]


# set size

In [151]:
def getCompaniesInMonth(month):
  compainesInMonth = []
  for tickerKey in companyInfoDict.keys():
    if month in companyInfoDict[tickerKey]:
      compainesInMonth.append(companyInfoDict[tickerKey][month])
  return compainesInMonth

In [152]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.rollingAverage is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeMOM = "Big"
    for company in companiesInMonth[median:]:
        company.sizeMOM = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeMOM ][-10:])

[]


In [153]:

    
    
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.rollingAverage is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("rollingAverage"), reverse=True)
    count = math.floor(len(companiesInMonth) * 0.3) #It might be better to round it up instead of down to increase valid data
    for company in companiesInMonth[0:count]:
        company.momentum = "High"
    for company in companiesInMonth[len(companiesInMonth) - count:]:
        company.momentum = "Low"
    for company in companiesInMonth[count:len(companiesInMonth) - count]:
        company.momentum = "Notural"

print ([x for x in getCompaniesInMonth(allMonths[-2]) if x.momentum is not None ][-10:])

[tickerKey:103155	 month:140003	 closeAdjust:680.0	 currentYield:-0.16666666666666663	 numberOfOutstandingShares:114518370483.0	 marketcap:80849969560998.0	 b2m:0.6337402775	 rollingAverage:-0.03401663695781341	 momentum:Notural	 sizeMOM:Big	 sizeHML:None	 sizeRMW:None	 sizeCMA:None
, tickerKey:103315	 month:140003	 closeAdjust:10820.0	 currentYield:0.08488986814483468	 numberOfOutstandingShares:2500000000.0	 marketcap:27007500000000.0	 b2m:3.9789218637	 rollingAverage:-0.06501226714057151	 momentum:Low	 sizeMOM:Big	 sizeHML:None	 sizeRMW:None	 sizeCMA:None
, tickerKey:103316	 month:140003	 closeAdjust:18745.0	 currentYield:-0.3444120837011866	 numberOfOutstandingShares:3000000000.0	 marketcap:56823000000000.0	 b2m:8.6749608558	 rollingAverage:-0.05712690122137463	 momentum:Notural	 sizeMOM:Big	 sizeHML:None	 sizeRMW:None	 sizeCMA:None
, tickerKey:103318	 month:140003	 closeAdjust:11528.0	 currentYield:0.13734747341582243	 numberOfOutstandingShares:3000000000.0	 marketcap:3420000000000

In [154]:
def avgTriple(first, second, third):
    avg = 0
    count = 0
    if first is not None:
        count += 1
        avg += first
    if second is not None:
        count += 1
        avg += second
    if third is not None:
        count += 1
        avg += third
    if count != 0:
        avg /= count
    return avg

# MomentomFactor factor (MOM)

In [155]:


MoMFactor = {}

for index,month in enumerate(allMonths[:-1]):
    # Get companies of corrent month
    companiesInMonth = getCompaniesInMonth(month)
    # Get companis for these filters SL SH BL BH
    SL = [company for company in companiesInMonth if company.sizeMOM == "Small" and company.momentum == "Low"]
    SH = [company for company in companiesInMonth if company.sizeMOM == "Small" and company.momentum == "High"]
    BL = [company for company in companiesInMonth if company.sizeMOM == "Big" and company.momentum == "Low"]
    BH = [company for company in companiesInMonth if company.sizeMOM == "Big" and company.momentum == "High"]
    # Get companies next month info
    BH_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in BH]
    BL_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in BL]
    SH_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in SH]
    SL_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in SL]
    # Get companies next month currentYield
    BH_average = valueWeightedAverage([x for x in BH_nextMonth if x is not None])
    BL_average = valueWeightedAverage([x for x in BL_nextMonth if x is not None])
    SH_average = valueWeightedAverage([x for x in SH_nextMonth if x is not None])
    SL_average = valueWeightedAverage([x for x in SL_nextMonth if x is not None])
    
    MoMFactor[month]=avgTriple(SH_average, BH_average,None) - avgTriple(SL_average, BL_average,None)

print(MoMFactor)


{138001: 0, 138002: 0, 138003: 0, 138004: 0, 138005: 0, 138006: 0, 138007: 0.050605533044555596, 138008: 0.039632358199331474, 138009: 0.057557775911981475, 138010: 0.012179434287274464, 138011: 0.11379667641635757, 138012: 0.030938173835893028, 138101: 0.11872634357341363, 138102: 0.08867768887857716, 138103: 0.13185507239267638, 138104: 0.06767217299891831, 138105: 0.014663454055563817, 138106: -0.011025297204854834, 138107: 0.039568039601327244, 138108: 0.044374178553870755, 138109: 0.04674525169824294, 138110: 0.035971968308430725, 138111: 0.012125443793257253, 138112: 0.0457545908672497, 138201: 0.09236209581892066, 138202: 0.1292485224160534, 138203: 0.201406882234963, 138204: 0.06837476147172283, 138205: 0.006134700795202008, 138206: -0.04758047471133903, 138207: 0.09673655856830277, 138208: -0.055082778745565386, 138209: 0.037962382672156444, 138210: -0.031856262461863315, 138211: -0.010243132654243297, 138212: -0.03723238540661818, 138301: 0.04566693180105387, 138302: 0.029798

# Book to market Calc

In [156]:
#  Fill all marketcap based on the last one
for companyTicker in companyTickerSet:
  for index,month in enumerate(allMonths):
    if month not in companyInfoDict[companyTicker]:
      firstData:CompanyInfo = companyInfoDict[companyTicker][list(companyInfoDict[companyTicker].keys())[0]]
      c = CompanyInfo(companyTicker,firstData.name,month,None,None,None,None,None,None, firstData.sourceSubsectorCode)
      if index > 0 and allMonths[index-1] in companyInfoDict[companyTicker]:
        c.marketcap = companyInfoDict[companyTicker][allMonths[index-1]].marketcap
        c.b2m = companyInfoDict[companyTicker][allMonths[index-1]].b2m
        c.inv = companyInfoDict[companyTicker][allMonths[index-1]].inv
        c.op = companyInfoDict[companyTicker][allMonths[index-1]].op
      companyInfoDict[companyTicker][month] = c
  # print(companyInfoDict[companyTicker][139712])

for month in sorted(companyInfoDict[1].keys())[:10]:
  print(companyInfoDict[1][month])

tickerKey:1	 month:138001	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

tickerKey:1	 month:138002	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

tickerKey:1	 month:138003	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

tickerKey:1	 month:138004	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 sizeMOM:None	 sizeHML:None	 sizeRMW:None	 sizeCMA:None

tickerKey:1	 month:138005	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 m

In [157]:
# This calculation are done manually; Enable this to make it automated.
"""
for companyTicker in companyTickerSet:
  lastB2MOfTheYear = None
  lastINVOfTheYear = None
  lastOpOfTheYear = None
  for month in allMonths:
    if month % 100 == 5:
      lastB2MOfTheYear = companyInfoDict[companyTicker][month].b2m
      lastINVOfTheYear = companyInfoDict[companyTicker][month].inv
      lastOpOfTheYear = companyInfoDict[companyTicker][month].op
    companyInfoDict[companyTicker][month].b2m = lastB2MOfTheYear     
    companyInfoDict[companyTicker][month].inv = lastINVOfTheYear     
    companyInfoDict[companyTicker][month].op = lastOpOfTheYear     
"""

'\nfor companyTicker in companyTickerSet:\n  lastB2MOfTheYear = None\n  lastINVOfTheYear = None\n  lastOpOfTheYear = None\n  for month in allMonths:\n    if month % 100 == 5:\n      lastB2MOfTheYear = companyInfoDict[companyTicker][month].b2m\n      lastINVOfTheYear = companyInfoDict[companyTicker][month].inv\n      lastOpOfTheYear = companyInfoDict[companyTicker][month].op\n    companyInfoDict[companyTicker][month].b2m = lastB2MOfTheYear     \n    companyInfoDict[companyTicker][month].inv = lastINVOfTheYear     \n    companyInfoDict[companyTicker][month].op = lastOpOfTheYear     \n'

In [158]:
# for month in sorted(companyInfoDict[1].keys())[:10]:
#   print(companyInfoDict[9][month])

In [159]:
# Book To market ranking

for index,month in enumerate(allMonths[:-1]):
  b2mMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].b2m == None or \
      companyInfoDict[companyTicker][month].marketcap == None or \
      findCompanyInfo(companyTicker,allMonths[index+1]).currentYield == None :
      continue
    b2mMonthList.append(companyInfoDict[companyTicker][month].b2m)
  if len(b2mMonthList) == 0 :
    continue
  b2mMonthList.sort()
  # print(len(b2mMonthList),int(len(b2mMonthList)*0.3))
  lowBookToMarket = b2mMonthList[int(len(b2mMonthList)*0.3)]
  highBookToMarket = b2mMonthList[int(-len(b2mMonthList)*0.3)]

  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].b2m == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue
    if companyInfoDict[companyTicker][month].b2m <= lowBookToMarket:
      companyInfoDict[companyTicker][month].BperMSize = "Low"
    elif companyInfoDict[companyTicker][month].b2m >= highBookToMarket:
      companyInfoDict[companyTicker][month].BperMSize = "High"
    else:
      companyInfoDict[companyTicker][month].BperMSize = "Neutral"



In [160]:
# RMW (Operational profit) ranking

for index,month in enumerate(allMonths[:-1]):
  dataMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].op == None or \
      companyInfoDict[companyTicker][month].marketcap == None or \
      findCompanyInfo(companyTicker,allMonths[index+1]).currentYield == None :
      continue
    dataMonthList.append(companyInfoDict[companyTicker][month].op)
  if len(dataMonthList) == 0 :
    continue
  dataMonthList.sort()
  # print(len(dataMonthList),int(len(dataMonthList)*0.3))
  weakValue = dataMonthList[int(len(dataMonthList)*0.3)]
  robustValue = dataMonthList[int(-len(dataMonthList)*0.3)]
  # if month == 139902:
  #   print(dataMonthList)
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].op == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue
    if companyInfoDict[companyTicker][month].op <= weakValue:
      companyInfoDict[companyTicker][month].RMWSize = "Weak"
    elif companyInfoDict[companyTicker][month].op >= robustValue:
      companyInfoDict[companyTicker][month].RMWSize = "Robust"
    else:
      companyInfoDict[companyTicker][month].RMWSize = "Neutral"

In [161]:

for companyTicker in companyTickerSet[:10]:
  print(companyInfoDict[companyTicker][139807].RMWSize)

Robust
Robust
Neutral
Robust
Robust
Weak
Neutral
Neutral
Weak
Weak


In [162]:
# CMA (Investment) ranking

for index,month in enumerate(allMonths[:-1]):
  dataMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].inv == None or \
      companyInfoDict[companyTicker][month].marketcap == None or \
      findCompanyInfo(companyTicker,allMonths[index+1]).currentYield == None :
      continue   
    dataMonthList.append(companyInfoDict[companyTicker][month].inv)
  if len(dataMonthList) == 0 :
    continue
  dataMonthList.sort()
  # print(len(dataMonthList),int(len(dataMonthList)*0.3))
  weakValue = dataMonthList[int(len(dataMonthList)*0.3)]
  robustValue = dataMonthList[int(-len(dataMonthList)*0.3)]
  # if month == 139902:
  #   print(dataMonthList)
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].inv == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue
    if companyInfoDict[companyTicker][month].inv <= weakValue:
      companyInfoDict[companyTicker][month].CMASize = "Conservative"
    elif companyInfoDict[companyTicker][month].inv >= robustValue:
      companyInfoDict[companyTicker][month].CMASize = "Aggressive"
    else:
      companyInfoDict[companyTicker][month].CMASize = "Neutral"

In [163]:
def getNextMonth(currentMonth):
  return allMonths[allMonths.index(currentMonth)+1]

def getNextMonthCompaniesHavingYeild(companyInfoList):
  response = []
  for companyInfo in companyInfoList:
    nextMoonth = companyInfoDict[companyInfo.tickerKey][getNextMonth(companyInfo.month)]
    if nextMoonth.currentYield != None:
      response.append(nextMoonth)
  return response


In [164]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.b2m is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeHML = "Big"
    for company in companiesInMonth[median:]:
        company.sizeHML = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeHML ][-10:])

[]


In [165]:
HMLFactor = {}
SMBBTMFactor = {}
for month in allMonths[:-1]:
  SH = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "High" and \
        companyInfoDict[companyTicker][month].sizeHML == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeHML == "Small"]
  SL = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "Low" and \
        companyInfoDict[companyTicker][month].sizeHML == "Small"]
  BH = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "High" and \
        companyInfoDict[companyTicker][month].sizeHML == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeHML == "Big"]
  BL = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "Low" and \
        companyInfoDict[companyTicker][month].sizeHML == "Big"]
  
  
  SH_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SH))
  SN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SN))
  SL_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SL))
  BH_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BH))
  BN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BN))
  BL_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BL))
  
  SMBBTMFactor[month] = avgTriple(SH_average,SN_average,SL_average) - avgTriple(BH_average, BN_average, BL_average)
  HMLFactor[month] = avgTriple(SH_average, BH_average,None)  - avgTriple(SL_average , BL_average,None)
for month in list(HMLFactor.keys())[:10]:
  print(HMLFactor[month])

0
0.0038989841984604437
0.02291386935525331
0.05640435648533565
-0.024684754966975808
0.0014765126165046712
0.019141134043749293
-0.002299456377792067
0.02195206636740205
-0.029414204569927545


In [166]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.op is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeRMW = "Big"
    for company in companiesInMonth[median:]:
        company.sizeRMW = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeRMW ][-10:])

[]


In [167]:
RMWFactor = {}
SMBRMWFactor = {}
for month in allMonths[:-1]:
  SR = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Robust" and \
        companyInfoDict[companyTicker][month].sizeRMW == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeRMW == "Small"]
  SW = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Weak" and \
        companyInfoDict[companyTicker][month].sizeRMW == "Small"]
  BR = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Robust" and \
        companyInfoDict[companyTicker][month].sizeRMW == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeRMW == "Big"]
  BW = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Weak" and \
        companyInfoDict[companyTicker][month].sizeRMW == "Big"]
  
  SR_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SR))
  SN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SN))
  SW_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SW))
  BR_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BR))
  BN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BN))
  BW_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BW))

  SMBRMWFactor[month] = avgTriple(SR_average , SN_average, SW_average) - avgTriple(BR_average, BN_average, BW_average)
  RMWFactor[month] = avgTriple(SR_average, BW_average,None) - avgTriple(SW_average, BW_average,None)
for month in list(RMWFactor.keys())[:10]:
  print(RMWFactor[month])

0
0.018465601505908022
0.00856511081428378
0.03858368974668194
0.0028669732955785704
0.01120282839688961
-0.01773046853530759
-0.017240285062923414
0.01767527712420763
-0.000212743716439384


In [168]:
import math
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.op is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.sizeCMA = "Big"
    for company in companiesInMonth[median:]:
        company.sizeCMA = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.sizeCMA ][-10:])

[]


In [169]:
CMAFactor = {}
SMBCMAFactor = {}
for month in allMonths[:-1]:
  SA = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Aggressive" and \
        companyInfoDict[companyTicker][month].sizeCMA == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeCMA == "Small"]
  SC = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Conservative" and \
        companyInfoDict[companyTicker][month].sizeCMA == "Small"]
  BA = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Aggressive" and \
        companyInfoDict[companyTicker][month].sizeCMA == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Neutral" and \
        companyInfoDict[companyTicker][month].sizeCMA == "Big"]
  BC = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Conservative" and \
        companyInfoDict[companyTicker][month].sizeCMA == "Big"]
  
  SA_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SA))
  SN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SN))
  SC_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(SC))
  BA_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BA))
  BN_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BN))
  BC_average = valueWeightedAverage(getNextMonthCompaniesHavingYeild(BC))

  SMBCMAFactor[month] = avgTriple(SA_average, SN_average, SC_average) - avgTriple(BA_average, BN_average, BC_average)
  CMAFactor[month] = avgTriple(SC_average, BC_average,None)  - avgTriple(SA_average, BA_average,None)
for month in list(CMAFactor.keys())[:10]:
  print(CMAFactor[month])

0
-0.022069992949631523
-0.00541160012636114
-0.018548747656179143
0.00930233880376085
3.6003457451663085e-05
0.030504184483581023
0.005630102351359842
0.00889257138108445
-0.014336779233293672


In [170]:
SMBFactor = {}
for month in SMBBTMFactor.keys():
  SMBFactor[month] = (SMBBTMFactor[month] + SMBCMAFactor[month] + SMBRMWFactor[month]) / 3


# Export company list to excel

In [171]:
# Example to excel function
exportCompanyListToExcel("calculations")

In [172]:
# Factors to excel
rows = []
for month in allMonths:
     rows.append([
       month,
       SMBFactor[month] if month in SMBFactor else None,
       HMLFactor[month] if month in HMLFactor else None,
       RMWFactor[month] if month in RMWFactor else None,
       CMAFactor[month] if month in CMAFactor else None,
       MoMFactor[month] if month in MoMFactor else None,
     ])
dfOut = pd.DataFrame(rows, columns=['month',"SMBFactor", "HMLFactor", "RMWFactor", "CMAFactor", "MoMFactor"])
dfOut.to_excel('Factors.xlsx', sheet_name='Factors')